In [12]:
import numpy as np
from keras import Sequential
from keras.src.callbacks import ModelCheckpoint, EarlyStopping
from keras.src.layers import Dense
from keras.src.utils import to_categorical
import plotly.graph_objects as go
import pandas as pd
from math import floor
from sklearn.metrics import confusion_matrix, accuracy_score
import plotly.express as exp

In [27]:
data = np.random.randint(0, 100, size=(5000, 5))

data

array([[23, 45, 55, 19, 16],
       [22,  2, 46, 74, 38],
       [51, 10, 27, 76, 12],
       ...,
       [86, 16, 92, 36, 68],
       [51, 22, 13, 13, 70],
       [27, 89, 52, 91, 36]])

In [30]:
temp = []

for i in data:
    record_sum = sum(i)

    if 0 <= record_sum < 100:
        temp.append([0])
    elif 100 <= record_sum < 200:
        temp.append([1])
    elif 200 <= record_sum < 300:
        temp.append([2])
    elif 300 <= record_sum < 400:
        temp.append([3])
    elif 400 <= record_sum:
        temp.append([4])

labels = np.array(temp)
exp.histogram(labels)

In [31]:
labels = to_categorical(labels, num_classes=5)

labels

array([[0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [39]:
model = Sequential()
model.add(Dense(units=32, input_shape=(5,)))
model.add(Dense(units=16))
model.add(Dense(units=16))
model.add(Dense(units=5, activation='softmax'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 32)                192       
                                                                 
 dense_12 (Dense)            (None, 16)                528       
                                                                 
 dense_13 (Dense)            (None, 16)                272       
                                                                 
 dense_14 (Dense)            (None, 5)                 85        
                                                                 
Total params: 1077 (4.21 KB)
Trainable params: 1077 (4.21 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [40]:
mc = ModelCheckpoint('model2.keras', monitor='val_accuracy', save_best_only=True, mode="max", verbose=1)
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=7)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(data, labels, epochs=50, validation_split=0.25, callbacks=[mc, es])

Epoch 1/50
111/118 [===========================>..] - ETA: 0s - loss: 2.8126 - accuracy: 0.4400
Epoch 1: val_accuracy improved from -inf to 0.52960, saving model to model2.keras
118/118 [==============================] - 2s 6ms/step - loss: 2.7268 - accuracy: 0.4448 - val_loss: 1.3449 - val_accuracy: 0.5296
Epoch 2/50
113/118 [===========================>..] - ETA: 0s - loss: 1.2323 - accuracy: 0.4851
Epoch 2: val_accuracy improved from 0.52960 to 0.55360, saving model to model2.keras
118/118 [==============================] - 1s 4ms/step - loss: 1.2316 - accuracy: 0.4851 - val_loss: 1.4454 - val_accuracy: 0.5536
Epoch 3/50
108/118 [==========================>...] - ETA: 0s - loss: 1.1654 - accuracy: 0.5029
Epoch 3: val_accuracy did not improve from 0.55360
118/118 [==============================] - 0s 4ms/step - loss: 1.1627 - accuracy: 0.5045 - val_loss: 1.1293 - val_accuracy: 0.5080
Epoch 4/50
109/118 [==========================>...] - ETA: 0s - loss: 1.0507 - accuracy: 0.5321
Epoch

In [41]:
def draw_history(history):
    fig1 = go.Figure()
    fig1.add_trace(go.Scatter(x=history.epoch, y=history.history['accuracy'], name="accuracy"))
    fig1.add_trace(go.Scatter(x=history.epoch, y=history.history['val_accuracy'], name="val_accuracy"))
    fig1.show()

    fig2 = go.Figure()
    fig2.add_trace(go.Scatter(x=history.epoch, y=history.history['loss'], name="loss"))
    fig2.add_trace(go.Scatter(x=history.epoch, y=history.history['val_loss'], name="val_loss"))
    fig2.show()


draw_history(history)

In [42]:
test_data = np.random.randint(0, 100, size=(2500, 5))

# model.save_weights("model_after_training.keras")
# model.load_weights("model2.keras")

predicts = model.predict(test_data, verbose=0)

classes = np.argmax(predicts, axis=1)

In [43]:
df = pd.DataFrame(columns=("record_sum", "class", "prediction", "error"))

for i in range(len(test_data)):
    s = sum(test_data[i])
    df.loc[len(df)] = [s, floor(s / 100), classes[i], classes[i] != floor(s / 100)]

df

,record_sum,class,prediction,error
0,276,2,2,False
1,235,2,2,False
2,192,1,2,True
3,345,3,3,False
4,177,1,1,False
...,...,...,...,...
2495,187,1,1,False
2496,357,3,3,False
2497,186,1,1,False
2498,206,2,1,True


In [25]:
exp.histogram(df["class"], range_x=[0, 4])

In [44]:
print(f"Accuracy: {round(accuracy_score(df['class'], df['prediction']) * 100, 2)} %")

cm = confusion_matrix(df["class"], df["prediction"])
fig = exp.imshow(cm,
                 text_auto=True,
                 x=["p0", "p1", "p2", "p3", "p4"],
                 y=["t0", "t1", "t2", "t3", "t4"],
                 )

fig.show()

Accuracy: 91.4 %
